In [1]:
import numpy as np
import sys

sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import tensorflow as tf

import aux_functions_stochastic

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_Oscillator import OscillatorDataHandler

from ann_framework import aux_functions

Using TensorFlow backend.


### Define keras model

In [2]:
"""

l2_lambda_regularization = 0.3

def StochasticModel_SN(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_shape=input_shape, activation='sigmoid', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, activation='sigmoid', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model
"""

"\n\nl2_lambda_regularization = 0.3\n\ndef StochasticModel_SN(input_shape):\n    #Create a sequential model\n    model = Sequential()\n    \n    #Add the layers for the model\n    model.add(Dense(20, input_shape=input_shape, activation='sigmoid', kernel_initializer='glorot_normal', \n                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), \n                    name='fc1'))\n    model.add(Dense(20, activation='sigmoid', kernel_initializer='glorot_normal', \n                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), \n                    name='fc2'))\n    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', \n                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))\n    \n    return model\n"

### Define tensorflow model

In [3]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y

def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), 
                         name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y

### Create  Model Keras

In [4]:
"""

def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001,beta_1=0.5)
    
    #optimizer = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models
    model = StochasticModel_SN(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model
    
"""

'\n\ndef get_compiled_model(shape, model_type=\'ann\'):\n    \n    K.clear_session()  #Clear the previous tensorflow graph\n    \n    #To test the model without randomness\n\n    \n    #Shared parameters for the models\n    optimizer = Adam(lr=0.001,beta_1=0.5)\n    \n    #optimizer = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)\n    \n    lossFunction = "mean_squared_error"\n    metrics = ["mse"]\n    model = None\n\n    #Create and compile the models\n    model = StochasticModel_SN(shape)\n    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)\n\n    return model\n    \n'

### Create Model TF

In [5]:
def first_order_central_finite_difference(f_x_delta_plus, f_x_delta_minus, delta):
    
    return (f_x_delta_plus - f_x_delta_minus)/(2*delta)

def second_order_central_finite_difference(f_x, f_x_delta_plus, f_x_delta_minus, delta):
    
    return (f_x_delta_minus - 2*f_x + f_x_delta_minus)/(delta**2)
    

In [6]:
def squared_residual_function(X, y, y_pred):
    
    k = 1
    c = 0.1
    D = 1
    r = 0
    
    deltas = tf.constant([0.1,0.1], name="deltas")
    
    deltas = tf.Print(deltas, [deltas, X], "que mamada\n")
    
    """
    delta_x = deltas[0]
    delta_y = deltas[1]
    
    nn_x = y_pred[0]
    nn_x_delta_plus = y_pred[1]
    nn_x_delta_minus = y_pred[2]
    nn_y_delta_plus = y_pred[3]
    nn_y_delta_minus = y_pred[4]
    
    nn_partial1_x = first_order_central_finite_difference(nn_x_delta_plus, nn_x_delta_minus, delta_x)
    nn_partial1_y = first_order_central_finite_difference(nn_y_delta_plus, nn_y_delta_minus, delta_y)
    nn_partial2_y = second_order_central_finite_difference(nn_x, nn_y_delta_plus, nn_y_delta_minus, delta_y)
    
    #r = y*nn_partial1_x + c - c*y*nn_partial1_y - k*x*nn_partial1_y + D*(nn_partial1_y**2 - nn_partial2_y)
    """
    
    return tf.losses.mean_squared_error(y, y_pred)
     
    
    

In [7]:
def tf_compiled_model(num_features, output_shape):
    
    tf.reset_default_graph()
    X, y = create_placeholders(num_features, output_shape)
    
    y_pred = tf_model(X)
    #cost = tf.losses.mean_squared_error(y, y_pred)
    cost = squared_residual_function(X, y, y_pred)
    reg_cost = tf.losses.get_regularization_loss()
    total_cost = cost + reg_cost
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001,beta1=0.5).minimize(total_cost)
    
    return {'X_placeholder':X, 'y_placeholder':y, 'y_pred':y_pred, 'cost':cost, 'total_cost':total_cost, 'optimizer':optimizer}

### Create Tunable Model Keras

In [8]:
"""

shape = (2, )

dhandler_stochastic = OscillatorDataHandler()


model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='keras', 
                                        data_handler=dhandler_stochastic)
tModel.model.summary()

"""

"\n\nshape = (2, )\n\ndhandler_stochastic = OscillatorDataHandler()\n\n\nmodel = get_compiled_model(shape, model_type='ann')\ntModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='keras', \n                                        data_handler=dhandler_stochastic)\ntModel.model.summary()\n\n"

### Create Tunable Model Tensorflow

In [9]:
dhandler_stochastic = OscillatorDataHandler()


model = tf_compiled_model(2, 1)
tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', 
                                        data_handler=dhandler_stochastic)

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to use tf.print in python 2.7, users must make sure to import
the following:

  `from __future__ import print_function`



### Load Data

In [10]:
#tModel.data_handler.data_scaler = None
#tModel.data_scaler = min_max_scaler

tModel.load_data(verbose=1, cross_validation_ratio=0.2, x=[0,0], delta_x=[5,5], n=[50,50])
#tModel.data_handler.print_data()
tModel.print_data()

Loading data. Cros-Validation ratio 0.2
Printing shapes

Training data (X, y)
(7200, 2)
(7200, 1)
Cross-Validation data (X, y)
(1800, 2)
(1800, 1)
Testing data (X, y)
(1000, 2)
(1000, 1)
Printing first 5 elements

Training data (X, y)
[[-3.88888889 -2.07070707]
 [ 3.58585859 -2.17171717]
 [ 4.7979798   1.76767677]
 [ 1.56565657 -2.87878788]
 [ 4.19191919  2.07070707]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Cross-Validation data (X, y)
[[-3.48484848 -5.        ]
 [ 4.39393939  4.7979798 ]
 [ 4.6969697   3.78787879]
 [-2.07070707  3.68686869]
 [ 2.67676768 -2.57575758]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Testing data (X, y)
[[-4.29292929  2.27272727]
 [-3.18181818  4.29292929]
 [ 4.19191919 -1.66666667]
 [-4.19191919  2.47474747]
 [-0.95959596  4.8989899 ]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Train model Keras

In [11]:
"""

tModel.epochs = 200
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

tModel.train_model(learningRate_scheduler=lrate, verbose=1)

"""

'\n\ntModel.epochs = 200\nlrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)\n\ntModel.train_model(learningRate_scheduler=lrate, verbose=1)\n\n'

### Train model Tensorflow

In [12]:
tModel.epochs = 200
lrate = LearningRateScheduler(aux_functions.step_decay)
minibatches_function_handle = aux_functions_stochastic.get_minibatches

deltas = [0.1, 0.1]
#tf.get_variable("deltas", initializer=deltas)

sess = tf.Session()

writer = tf.summary.FileWriter('./graphs', sess.graph)

tModel.train_model(tf_session=sess, get_minibatches_function_handle=minibatches_function_handle, 
                   verbose=1, deltas=deltas)

Epoch: 0001 cost_reg= 11.735748736 cost= 0.903924761
Epoch: 0002 cost_reg= 10.379637019 cost= 0.208065661
Epoch: 0003 cost_reg= 9.578577805 cost= 0.093574203
Epoch: 0004 cost_reg= 8.872437986 cost= 0.059824997
Epoch: 0005 cost_reg= 8.215274302 cost= 0.045167493
Epoch: 0006 cost_reg= 7.604796918 cost= 0.036156503
Epoch: 0007 cost_reg= 7.033651479 cost= 0.030200790
Epoch: 0008 cost_reg= 6.490396945 cost= 0.024286012
Epoch: 0009 cost_reg= 5.976389599 cost= 0.020563973
Epoch: 0010 cost_reg= 5.493213844 cost= 0.016338221
Epoch: 0011 cost_reg= 5.041775290 cost= 0.014419000
Epoch: 0012 cost_reg= 4.620947011 cost= 0.011843789
Epoch: 0013 cost_reg= 4.229043738 cost= 0.010077109
Epoch: 0014 cost_reg= 3.866402070 cost= 0.008716067
Epoch: 0015 cost_reg= 3.526670758 cost= 0.007311991
Epoch: 0016 cost_reg= 3.208980576 cost= 0.006488359
Epoch: 0017 cost_reg= 2.917697303 cost= 0.006230074
Epoch: 0018 cost_reg= 2.654797061 cost= 0.005937626
Epoch: 0019 cost_reg= 2.415947564 cost= 0.005601093
Epoch: 002

Epoch: 0161 cost_reg= 0.011779122 cost= 0.000000000
Epoch: 0162 cost_reg= 0.011804059 cost= 0.000000000
Epoch: 0163 cost_reg= 0.011783525 cost= 0.000000000
Epoch: 0164 cost_reg= 0.011812355 cost= 0.000000000
Epoch: 0165 cost_reg= 0.011794279 cost= 0.000000000
Epoch: 0166 cost_reg= 0.011819305 cost= 0.000000000
Epoch: 0167 cost_reg= 0.011798645 cost= 0.000000000
Epoch: 0168 cost_reg= 0.011827112 cost= 0.000000000
Epoch: 0169 cost_reg= 0.011808633 cost= 0.000000000
Epoch: 0170 cost_reg= 0.011833730 cost= 0.000000000
Epoch: 0171 cost_reg= 0.011812943 cost= 0.000000000
Epoch: 0172 cost_reg= 0.011841080 cost= 0.000000000
Epoch: 0173 cost_reg= 0.011822226 cost= 0.000000000
Epoch: 0174 cost_reg= 0.011847376 cost= 0.000000000
Epoch: 0175 cost_reg= 0.011826464 cost= 0.000000000
Epoch: 0176 cost_reg= 0.011854299 cost= 0.000000000
Epoch: 0177 cost_reg= 0.011835101 cost= 0.000000000
Epoch: 0178 cost_reg= 0.011860288 cost= 0.000000000
Epoch: 0179 cost_reg= 0.011839251 cost= 0.000000000
Epoch: 0180 